In [3]:
import pandas as pd 
from tqdm import tqdm 
tqdm.pandas()

restau = pd.read_json('yelp_restaurants.json')
review = pd.read_json('yelp_review.json')
user = pd.read_json('yelp_user.json')

In [4]:
user_clean = user[['user_id','name','review_count','useful','cool','fans']]
user_clean = user_clean.rename(columns={'user_id':'user_id:ID(User)'})
review_clean = review[['review_id','stars','useful','cool','text']]
review_clean['stars'] = list(map(int,review_clean['stars']))
review_clean = review_clean.rename(columns={'review_id':'review_id:ID(Review)'})
restau_clean = restau[['business_id','name','city']]
restau_clean = restau_clean.rename(columns={'business_id':'business_id:ID(Restaurant)'})

C:\Users\erber\AppData\Local\Temp\ipykernel_11700\1449736295.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_clean['stars'] = list(map(int,review_clean['stars']))


In [5]:
text = """coucou "machin" """
print(text)
text.replace("\"","\"\"")

coucou "machin" 


'coucou ""machin"" '

In [6]:
#cleaning des string de reviews 

for text in review_clean.text.values:
    c = "\""
    if c in text:
        # print(text)
        text = text.replace(c,"\"\"")
        text = "\""+text+"\""
        # print(text)

def add_truc_text(texte):
    c = "\""
    if c in texte:
        texte = texte.replace(c,"\"\"")
    texte = "\""+texte+"\""
    return texte

review_clean['text'].progress_apply(add_truc_text)    

100%|██████████| 49150/49150 [00:00<00:00, 502813.56it/s]


0        "This is the place for authentic Indian street...
1        "Food was great. But very disappointed with th...
2        "One of the few diners in North Wilmington, an...
3        "Yeah, I know what you're thinking. One Wawa s...
4        "My boyfriend and I have lived in trolley squa...
                               ...                        
49145    "Take out on a Saturday night around 9pm. Staf...
49146    "I loved the atmosphere! Clean place and good ...
49147    "We get takeout here pretty regularly, and hav...
49148    "Mia and her staff did an AMAZING job on my da...
49149    "Originally gave this a 4 star, but I knew I w...
Name: text, Length: 49150, dtype: object

In [7]:
friendship_dic = {} #user 1 vers user 2
friendship_dic[':START_ID(User)'] = {}
friendship_dic[':END_ID(User)'] = {}
id = 1
for i,r in user.iterrows():
    for friend in r['friends']:
        friendship_dic[':START_ID(User)'][id]= r['user_id']
        friendship_dic[':END_ID(User)'][id]= friend

        id+=1

friendship = pd.DataFrame.from_dict(friendship_dic)

In [8]:
friendship

,:START_ID(User),:END_ID(User)
1,IpLRJY4CP3fXtlEd8Y4GFQ,y3Wtx1pOvTiqsJRkjceXiw
2,IpLRJY4CP3fXtlEd8Y4GFQ,_BcWyKQL16ndpBdggh2kNA
3,IpLRJY4CP3fXtlEd8Y4GFQ,lRRuTimITgwzoXLIM3g9qw
4,IpLRJY4CP3fXtlEd8Y4GFQ,z02XKjGqJMONH59_lUHlTQ
5,IpLRJY4CP3fXtlEd8Y4GFQ,47K1MpOBhigOvjuBcDbm7Q
...,...,...
42772,sBrBqsyzGkvAwa6sGk9q1Q,nMtldMTrGbjejbCXJEqbmQ
42773,kEC-cWeQZLLHgkVZBN-1og,iFYykW2hxVbZcbzcviZp0A
42774,kEC-cWeQZLLHgkVZBN-1og,7tqBqky6u5IrJrV_g6S2Sg
42775,NUfGzTtNM3ZX3OKq8ytpNw,88S8cjbVq7joeIjYgYDwCw


In [9]:
write_dic = {} #user vers review
write_dic[':START_ID(User)'] = {}
write_dic[':END_ID(Review)'] = {}
id = 1
for i,r in review.iterrows():
    write_dic[':START_ID(User)'][id]= r['user_id']
    write_dic[':END_ID(Review)'][id]= r['review_id']
    id+=1

wrote = pd.DataFrame.from_dict(write_dic)

In [10]:
wrote

,:START_ID(User),:END_ID(Review)
1,8rrmvJ7W3ujplXuOhjE2Hg,2xlNGuN8naQTHO3_xY0lGw
2,TRFTDMJegE6DituEKBpEKQ,_Dkml_CfMMb_wWeGpFtczg
3,glO-c9FC0RaY2GyiamCQFA,EmLTCw2p9BhOwnzHFXgGaQ
4,cFaHx-FkCnzdfF8BccukIg,PlYifdeyiixRZkymbUNqVQ
5,hefsTMvc-iSBsY_Nfa8WCg,PtUd264rjqWuEzL89nf64w
...,...,...
49146,508Y2A3yYQOyGaMDUYFXQQ,nWGqiW9gSFeb4jFTw2_guA
49147,Eviotci6ld8KT8QmjfW2tw,hK_2Z2XFGz7H3Mdl-t0Ubg
49148,NBKyKmBWssn8Weq-H-sCRQ,nz-OnQGnVIyy-gs7xVSJZQ
49149,UP9dlqXxqJ2o5nfN--Fn2Q,XPrmT_RGh3t7uq9NYhcG7g


In [11]:
critic_dic = {} #de review à resto
critic_dic[':START_ID(Review)'] = {}
critic_dic[':END_ID(Restaurant)'] = {}
# critic_dic['text'] = {}
id = 1
for i,r in review.iterrows():
    critic_dic[':START_ID(Review)'][id]= r['review_id']
    critic_dic[':END_ID(Restaurant)'][id]= r['business_id']
    # critic_dic['text'][id]= r['text']
    id+=1

critics = pd.DataFrame.from_dict(critic_dic)

In [12]:
critics

,:START_ID(Review),:END_ID(Restaurant)
1,2xlNGuN8naQTHO3_xY0lGw,yDiWYpuzRxcTgSi_O4JZpQ
2,_Dkml_CfMMb_wWeGpFtczg,W6S482dRQ_9nCcLjfv8VNg
3,EmLTCw2p9BhOwnzHFXgGaQ,OlOVRlcEstxKK_6hMjy63A
4,PlYifdeyiixRZkymbUNqVQ,DAcqRGI_u8yNCcUIczfNxA
5,PtUd264rjqWuEzL89nf64w,5IFqqWJTaPNoY2ZgRlX73w
...,...,...
49146,nWGqiW9gSFeb4jFTw2_guA,tILA8ctsD_OZIcM2IfSyQA
49147,hK_2Z2XFGz7H3Mdl-t0Ubg,riwJWxj4850tGBhRahDt7g
49148,nz-OnQGnVIyy-gs7xVSJZQ,YNWEM2NVocUl6d74yRvpug
49149,XPrmT_RGh3t7uq9NYhcG7g,iWFK7U-wBR-68X0VFrSkNA


In [13]:
import json
import re

amb_list = []
for i,r in restau.iterrows():
    if r['attributes'] is not None:
        attr = r['attributes']
        if 'Ambience' in attr.keys():
            # print(attr['Ambience'])
            m = re.findall("'[a-zA-Z]*'",attr['Ambience'])
            for elt in m:
                elt = elt.replace("\'","")
                if elt not in amb_list:
                    amb_list.append(elt)

amb_list

['divey',
 'hipster',
 'casual',
 'touristy',
 'trendy',
 'intimate',
 'romantic',
 'classy',
 'upscale']

In [14]:
amb_df = pd.DataFrame(columns=['ambience_id:ID(Ambience)','ambience_name'])
id = 1
for amb in amb_list:
    amb_df = amb_df.append({'ambience_id:ID(Ambience)':id,'ambience_name':amb},ignore_index=True)
    id+=1
amb_df

C:\Users\erber\AppData\Local\Temp\ipykernel_11700\3832293667.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  amb_df = amb_df.append({'ambience_id:ID(Ambience)':id,'ambience_name':amb},ignore_index=True)
C:\Users\erber\AppData\Local\Temp\ipykernel_11700\3832293667.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  amb_df = amb_df.append({'ambience_id:ID(Ambience)':id,'ambience_name':amb},ignore_index=True)
C:\Users\erber\AppData\Local\Temp\ipykernel_11700\3832293667.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  amb_df = amb_df.append({'ambience_id:ID(Ambience)':id,'ambience_name':amb},ignore_index=True)
C:\Users\erber\AppData\Local\Temp\ipykernel_11700\3832293667.py:4: FutureWarning: The frame.append method is dep

,ambience_id:ID(Ambience),ambience_name
0,1,divey
1,2,hipster
2,3,casual
3,4,touristy
4,5,trendy
5,6,intimate
6,7,romantic
7,8,classy
8,9,upscale


In [15]:
amb_ids = {}
for i,r in amb_df.iterrows():
    amb_ids[r[1]]=r[0]
amb_ids

{'divey': 1,
 'hipster': 2,
 'casual': 3,
 'touristy': 4,
 'trendy': 5,
 'intimate': 6,
 'romantic': 7,
 'classy': 8,
 'upscale': 9}

In [16]:
amb_dict = {} #user 1 vers user 2
amb_dict[':START_ID(Restaurant)'] = {}
amb_dict[':END_ID(Ambience)'] = {}
id = 1
for i,r in restau.iterrows():
    amb_list_restau = []
    if r['attributes'] is not None:
        attr = r['attributes']
        if 'Ambience' in attr.keys():
            # print(attr['Ambience'])
            m = re.findall("'[a-zA-Z]*': [a-zA-Z]*",attr['Ambience'])
            for elt in m:
                a,b = elt.split(':')
                a = a.replace("\'","")
                if a not in amb_list_restau and 'True' in b:
                    amb_list_restau.append(a)
    for amb in amb_list_restau:
        amb_dict[':START_ID(Restaurant)'][id]= r['business_id']
        amb_dict[':END_ID(Ambience)'][id]= amb_ids[amb]

        id+=1

ambiances_restau = pd.DataFrame.from_dict(amb_dict)

In [17]:
cats_list = restau.categories.unique()
cat_names = []

for cats in cats_list:
    temp = cats.split(', ')
    for elt in temp:
        if elt not in cat_names:
            cat_names.append(elt)

len(cat_names) 

192

In [18]:
cat_df = pd.DataFrame(columns=['category_id:ID(Category)','category_name'])
id = 1
for cat in cat_names:
    cat_df = cat_df.append({'category_id:ID(Category)':id,'category_name':cat},ignore_index=True)
    id+=1
cat_df

cat_ids = {}
for i,r in cat_df.iterrows():
    cat_ids[r[1]]=r[0]
cat_ids

C:\Users\erber\AppData\Local\Temp\ipykernel_11700\2719432253.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cat_df = cat_df.append({'category_id:ID(Category)':id,'category_name':cat},ignore_index=True)
C:\Users\erber\AppData\Local\Temp\ipykernel_11700\2719432253.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cat_df = cat_df.append({'category_id:ID(Category)':id,'category_name':cat},ignore_index=True)
C:\Users\erber\AppData\Local\Temp\ipykernel_11700\2719432253.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cat_df = cat_df.append({'category_id:ID(Category)':id,'category_name':cat},ignore_index=True)
C:\Users\erber\AppData\Local\Temp\ipykernel_11700\2719432253.py:4: FutureWarning: The frame.append method is dep

{'Mexican': 1,
 'Restaurants': 2,
 'Arts & Entertainment': 3,
 'Event Planning & Services': 4,
 'Italian': 5,
 'Gluten-Free': 6,
 'Nightlife': 7,
 'Lounges': 8,
 'Vegan': 9,
 'Bars': 10,
 'Caterers': 11,
 'Vegetarian': 12,
 'Pizza': 13,
 'Vietnamese': 14,
 'Asian Fusion': 15,
 'American (New)': 16,
 'Breakfast & Brunch': 17,
 'Thai': 18,
 'Middle Eastern': 19,
 'Japanese': 20,
 'Sushi Bars': 21,
 'Noodles': 22,
 'Food': 23,
 'Bakeries': 24,
 'Delis': 25,
 'Sandwiches': 26,
 'American (Traditional)': 27,
 'Breweries': 28,
 'Steakhouses': 29,
 'Seafood': 30,
 'Coffee & Tea': 31,
 'Fast Food': 32,
 'Burgers': 33,
 'Diners': 34,
 'Salad': 35,
 'Chicken Wings': 36,
 'Chicken Shop': 37,
 'Caribbean': 38,
 'Automotive': 39,
 'Convenience Stores': 40,
 'Gas Stations': 41,
 'Beer Bar': 42,
 'Cafes': 43,
 'Pubs': 44,
 'Brasseries': 45,
 'Cheesesteaks': 46,
 'Financial Services': 47,
 'Banks & Credit Unions': 48,
 'Indian': 49,
 'Soul Food': 50,
 'Southern': 51,
 'Dive Bars': 52,
 'Tex-Mex': 53,


In [19]:
cat_dict = {} #user 1 vers user 2
cat_dict[':START_ID(Restaurant)'] = {}
cat_dict[':END_ID(Category)'] = {}
id = 1
for i,r in restau.iterrows():
    cat_list_restau = r['categories'].split(', ')
    
    for cat in cat_list_restau:
        cat_dict[':START_ID(Restaurant)'][id]= r['business_id']
        cat_dict[':END_ID(Category)'][id]= cat_ids[cat]

        id+=1

cat_restau = pd.DataFrame.from_dict(cat_dict)

In [20]:
friendship.to_csv('csv/friendship.csv',index=False)
wrote.to_csv('csv/wrote.csv',index=False)
critics.to_csv('csv/critics.csv',index=False)
user_clean.to_csv('csv/users.csv',index=False)
review_clean.to_csv('csv/reviews.csv',index=False)
restau_clean.to_csv('csv/restaurants.csv',index=False)
amb_df.to_csv('csv/ambiences.csv',index=False)
ambiances_restau.to_csv('csv/has_ambience.csv',index=False)
cat_df.to_csv('csv/categories.csv',index=False)
cat_restau.to_csv('csv/has_category.csv',index=False)